In [111]:
from random import random
from random import seed
seed(1)

import pandas as pd
import numpy as np

In [147]:
# Config
MFRAC = 0.5
# Initial guess
ni = np.array([0.00027, 4.0, 0.063, 1.215, 0.145])
delta_n = np.array([0.0001, 0.001, 0.0001, 0.0001, 0.0001])
# Mutation step
mutation = np.array([0.09, 0.09, 0.09, 0.09, 0.09])

In [156]:
# Read data
ydata = pd.read_csv("./data/data1.csv")

ND = ydata.shape[0]        #number of data vectors in array
DI = ydata.shape[1]        #number of data items in vector
NS = ND                    #total number of DNA strands

# log
lydata = ydata.copy()
lydata = np.log(lydata + 1e-12)

# Initializing n
n = np.array([(ni + [random() for i in range(DI)] * delta_n) for i in range(ND)] )  # 45x5 randomized initial value

In [159]:
# Functions needed
# Error and Selection
def selection(n, lydata, MFRAC=MFRAC):
    # Error calculation
    Ferr = -lydata["HeatFlux"] + np.log(n[:, 0]) + n[:, 1] * lydata["Superheat"] + n[:, 2] * lydata["g"]
    aFerr = np.abs(Ferr) / np.abs(lydata["HeatFlux"])
    aFerrMean = np.mean(aFerr)
    aFerrMedian = np.median(aFerr)
    
    # Sorting
    sorted_id = np.argsort(aFerr)
    aFerr[:] = aFerr[sorted_id]
    sorted_n[:] = n[sorted_id]
    
    # Selection
    clim = MFRAC * aFerrMedian
    nkeep = np.searchsorted(aFerr, clim)
    return sorted_n, nkeep

# Mating
def mating(sorted_n, nkeep):
    if nkeep == sorted_n.shape[0]:
        print("NOT MATING")
        return sorted_n
    
    for row in sorted_n[nkeep:, :]:
        # Randomly choose 2 parents
        parents = [sorted_n[np.random.randint(0, nkeep+1)] for i in range(2)]
        
        # Choose which parent to inherit from
        # inherit[i] = 0 or 1
        inherit = [np.random.randint(0, 2) for i in range(DI)]
        r = [parents[inherit[i]][i] for i in range(DI)]
        
        # Mutation
        mut_rate = [(2*(0.5-np.random.rand())) for i in range(DI)]
        row[:] = r * (1 + mut_rate * mutation)
    return sorted_n
        
        
    

In [161]:
# Main
for i in range(6000):
    sorted_n, nkeep = selection(n, lydata)
    n = mating(sorted_n, nkeep)